# hp_file_extractor

In [102]:
DIRECTORY = '/host/dissertation/trained_models/1698862739/'

In [103]:
def q_hp_df_transform(df, sort_by='med(test)'):
    df = df[~df['hp_config'].str.contains('selu')]
    
    df[['topologia sieci', 'funkcja aktywacji']] = df['hp_config'].str.rsplit('-', n=1, expand=True)
    df.rename(columns={'val_acc-med': 'med(test)', 'train_acc-max': 'max(train)', 'val_acc-std': 'std(test)'}, inplace=True)
    df = df[['topologia sieci', 'funkcja aktywacji', 'max(train)', 'med(test)', 'std(test)']]

    df['topologia sieci'] = df['topologia sieci'].str.replace('-','x')
    df['funkcja aktywacji'] = df['funkcja aktywacji'].str.replace('_','\_')
    
    return df.sort_values(sort_by, ascending=False)

## q_hp

In [104]:
import pandas as pd

raw_q_hp_df = pd.read_csv(DIRECTORY + 'q_hp.csv')
print(len(raw_q_hp_df))
raw_q_hp_df

54


,hp_config,time,val_acc-avg,val_acc-med,val_acc-std,train_acc-max,train_acc-min
0,1-32-pol_tanh,18:21:35,0.500459,0.499694,0.008469,0.528539,0.514767
1,1-32-pol_sigmoid,18:23:55,0.496053,0.496022,0.008248,0.522724,0.510941
2,1-32-pol_selu,18:26:15,0.496695,0.497858,0.009290,0.514461,0.487070
3,1-32-cart_sigmoid,18:28:28,0.500979,0.500306,0.008025,0.535425,0.516756
4,1-32-cart_relu,18:30:42,0.898592,0.899939,0.030378,0.931140,0.922571
5,1-32-cart_tanh,18:32:56,0.507987,0.509180,0.009417,0.526090,0.507269
6,1-64-pol_tanh,18:35:19,0.504927,0.506732,0.009409,0.519204,0.501607
7,1-64-pol_sigmoid,18:37:42,0.495716,0.500306,0.011668,0.525784,0.511400
8,1-64-pol_selu,18:40:06,0.502326,0.504896,0.011262,0.512777,0.477429
9,1-64-cart_sigmoid,18:42:22,0.500979,0.503672,0.011185,0.538332,0.525325


In [111]:
transformed_df = q_hp_df_transform(raw_q_hp_df)
print(len(transformed_df))
transformed_df

45


/tmp/ipykernel_90/3615832989.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['topologia sieci', 'funkcja aktywacji']] = df['hp_config'].str.rsplit('-', n=1, expand=True)
/tmp/ipykernel_90/3615832989.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['topologia sieci', 'funkcja aktywacji']] = df['hp_config'].str.rsplit('-', n=1, expand=True)
/tmp/ipykernel_90/3615832989.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

,topologia sieci,funkcja aktywacji,max(train),med(test),std(test)
49,3x96,pol\_sigmoid,0.955317,0.955324,0.041579
25,2x64,pol\_sigmoid,0.958378,0.948592,0.015955
43,3x64,pol\_sigmoid,0.959602,0.948592,0.038028
48,3x96,pol\_tanh,0.958990,0.948286,0.016461
37,3x32,pol\_sigmoid,0.949809,0.945532,0.034321
30,2x96,pol\_tanh,0.957766,0.940024,0.025933
24,2x64,pol\_tanh,0.960673,0.939718,0.017398
31,2x96,pol\_sigmoid,0.950421,0.939718,0.014926
18,2x32,pol\_tanh,0.949809,0.932681,0.019918
19,2x32,pol\_sigmoid,0.941086,0.929621,0.018827


In [120]:
import pandas as pd
import numpy as np
import os

TAIL_SIZE = 25

hp_file_rows = []
history_filenames = list(filter(lambda filename: 'history' in filename, os.listdir(DIRECTORY)))
for history_filename in history_filenames:
    
    history_df = pd.read_csv(DIRECTORY + history_filename)
    val_accuracy_tail = history_df['val_accuracy'][-TAIL_SIZE:]
    accuracy_tail = history_df['accuracy'][-TAIL_SIZE:]
    
    new_row = {'hp_config':history_filename.rsplit('-', 1)[0],'val_acc-avg':np.average(val_accuracy_tail),'val_acc-med':np.median(val_accuracy_tail),'val_acc-std':np.std(val_accuracy_tail),'train_acc-max':np.max(accuracy_tail),'train_acc-min':np.min(accuracy_tail)}
    hp_file_rows.append(new_row)

hp_file_columns = ['hp_config','time','val_acc-avg','val_acc-med','val_acc-std','train_acc-max','train_acc-min']
q_hp_df = pd.DataFrame(hp_file_rows, columns=hp_file_columns)
q_hp_df = q_hp_df_transform(q_hp_df)
q_hp_df

/tmp/ipykernel_90/3615832989.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['topologia sieci', 'funkcja aktywacji']] = df['hp_config'].str.rsplit('-', n=1, expand=True)
/tmp/ipykernel_90/3615832989.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['topologia sieci', 'funkcja aktywacji']] = df['hp_config'].str.rsplit('-', n=1, expand=True)
/tmp/ipykernel_90/3615832989.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

,topologia sieci,funkcja aktywacji,max(train),med(test),std(test)
37,3x96,pol\_sigmoid,0.955317,0.955936,0.038896
43,2x64,pol\_sigmoid,0.958378,0.951040,0.015628
29,3x64,pol\_sigmoid,0.961285,0.945532,0.036131
47,3x32,pol\_sigmoid,0.949809,0.943696,0.039232
17,2x64,pol\_tanh,0.960673,0.942472,0.016754
38,3x96,pol\_tanh,0.958990,0.941860,0.022858
28,2x96,pol\_sigmoid,0.950421,0.940636,0.013564
51,2x96,pol\_tanh,0.957766,0.937576,0.023368
20,2x32,pol\_tanh,0.949809,0.932069,0.018884
8,2x32,pol\_sigmoid,0.941086,0.930845,0.017599


In [113]:
print(q_hp_df.to_latex(index=False))

\begin{tabular}{llrrr}
\toprule
topologia sieci & funkcja aktywacji & max(train) & med(test) & std(test) \\
\midrule
3x96 & pol\_sigmoid & 0.955777 & 0.952264 & 0.034456 \\
3x64 & pol\_sigmoid & 0.962357 & 0.950428 & 0.030291 \\
2x64 & pol\_sigmoid & 0.958378 & 0.946450 & 0.018809 \\
3x32 & pol\_sigmoid & 0.949809 & 0.944002 & 0.034829 \\
2x96 & pol\_sigmoid & 0.950421 & 0.941860 & 0.012459 \\
3x96 & pol\_tanh & 0.958990 & 0.941248 & 0.021907 \\
2x64 & pol\_tanh & 0.960673 & 0.940942 & 0.014900 \\
2x96 & pol\_tanh & 0.957766 & 0.939718 & 0.023327 \\
2x32 & pol\_sigmoid & 0.941086 & 0.930233 & 0.022880 \\
2x32 & pol\_tanh & 0.949809 & 0.929621 & 0.017346 \\
2x64 & cart\_relu & 0.952257 & 0.928703 & 0.028905 \\
3x64 & pol\_tanh & 0.947360 & 0.926867 & 0.032479 \\
1x64 & cart\_relu & 0.938791 & 0.925031 & 0.037499 \\
3x96 & cart\_relu & 0.959296 & 0.922277 & 0.022435 \\
3x32 & cart\_relu & 0.955317 & 0.920441 & 0.026247 \\
1x96 & cart\_relu & 0.946136 & 0.918605 & 0.031922 \\
2x96 & cart\